In [37]:
import pandas as pd


In [38]:
glassdoor_file = 'resources/glassdoor.csv'
df = pd.read_csv(glassdoor_file)
drop_col = df.columns
drop_col

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn',
       'spark', 'aws', 'excel'],
      dtype='object')

In [39]:
b = ['Job Title', 'Job Description', 'Rating', 'Location', 'Headquarters', 'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors', 'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary', 'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn', 'spark', 'aws', 'excel']
df.drop(axis=1, columns=b, inplace=True)
df.head()

,Salary Estimate,Company Name
0,$53K-$91K (Glassdoor est.),Tecolote Research\n3.8
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System\n3.4
2,$80K-$90K (Glassdoor est.),KnowBe4\n4.8
3,$56K-$97K (Glassdoor est.),PNNL\n3.8
4,$86K-$143K (Glassdoor est.),Affinity Solutions\n2.9


In [40]:
df['Company Name'].unique

<bound method Series.unique of 0                          Tecolote Research\n3.8
1      University of Maryland Medical System\n3.4
2                                    KnowBe4\n4.8
3                                       PNNL\n3.8
4                         Affinity Solutions\n2.9
                          ...                    
737                                      GSK\n3.9
738                               Eventbrite\n4.4
739           Software Engineering Institute\n2.6
740                             Numeric, LLC\n3.2
741             Riverside Research Institute\n3.6
Name: Company Name, Length: 742, dtype: object>

In [41]:
df.shape

(742, 2)

In [47]:
df['Company Name'].str.rstrip('\n')

0                          Tecolote Research\n3.8
1      University of Maryland Medical System\n3.4
2                                    KnowBe4\n4.8
3                                       PNNL\n3.8
4                         Affinity Solutions\n2.9
                          ...                    
737                                      GSK\n3.9
738                               Eventbrite\n4.4
739           Software Engineering Institute\n2.6
740                             Numeric, LLC\n3.2
741             Riverside Research Institute\n3.6
Name: Company Name, Length: 742, dtype: object